In [ ]:
 # Import necessary libraries
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM, Dropout
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load and preprocess data
data = pd.read_csv("Epileptic Seizure Recognition.csv")
x = data.values[:, 1:-1].astype(np.float32)
y = data['y']
y_categorical = to_categorical(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(x, y_categorical, test_size=0.2, random_state=42)

# Reshape data for LSTM
x_train = X_train.reshape(-1, 178, 1)
x_test = X_test.reshape(-1, 178, 1)

# Visualize some of the data
plt.figure(figsize=(12, 8))
sns.lineplot(x=range(len(x[1])), y=x[1], label='1')
sns.lineplot(x=range(len(x[7])), y=x[7], label='2')
sns.lineplot(x=range(len(x[12])), y=x[12], label='3')
sns.lineplot(x=range(len(x[0])), y=x[0], label='4')
sns.lineplot(x=range(len(x[2])), y=x[2], label='5')

plt.xlabel('X-axis label')
plt.ylabel('Y-axis label')
plt.title('Data Visualization')
plt.legend()
plt.show()

# Build the LSTM model
model = Sequential()
model.add(LSTM(56, input_shape=(178, 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(56))
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Activation('tanh'))
model.add(Dense(5))  # Assuming 5 classes based on y
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x_train, y_train, epochs=10, batch_size=100, verbose=2, validation_data=(x_test, y_test))

# Predict the results
ypred = model.predict(x_test)

# Convert predicted and actual values back to binary class labels (seizure vs non-seizure)
yp = np.argmax(ypred, axis=1)
yo = np.argmax(y_test, axis=1)

# Conversion of classes for seizure detection
yo = (yo != 1).astype(int)  # Convert all non-seizure classes to 0, seizure class remains 1
yp = (yp != 1).astype(int)  # Similarly convert predictions

# Calculate accuracy
accuracy = accuracy_score(yo, yp)
print(f"Accuracy: {accuracy}")


 